# Experiment Notebook: {Descriptive Experiment Keywords}

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to...

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [1]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

time: 55.4 s (started: 2023-01-12 23:00:34 +08:00)


In [2]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

time: 47.4 ms (started: 2023-01-12 23:01:33 +08:00)


In [3]:
# Import experiment templates
import experiments.templates.polygon_time_domain_analysis as time_domain_analysis
import experiments.templates.polygon_cumulative_yield_analysis as cumulative_yield_analysis
import experiments.templates.polygon_cumulative_yield_analysis2 as cumulative_yield_analysis2

time: 1.35 s (started: 2023-01-12 23:01:35 +08:00)


In [4]:
# Inspect experiment template
display_code(time_domain_analysis)

"""
# Time Domain Analysis

Executes a time-domain simulation over a long time period of 3 years,
over all Ethereum network upgrade stages.
"""

import copy

import model.constants as constants
from model.stochastic_processes import create_stochastic_process_realizations
from model.types import Stage
from experiments.default_experiment import experiment


# Make a copy of the default experiment to avoid mutation
experiment = copy.deepcopy(experiment)

DELTA_TIME = constants.epochs_per_day  # epochs per timestep
SIMULATION_TIME_MONTHS = 12*3  # number of months
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME


# Override default experiment Simulation and System Parameters related to timing
experiment.simulations[0].timesteps = TIMESTEPS
experiment.simulations[0].model.params.update({"dt": [DELTA_TIME]})

time: 44.5 ms (started: 2023-01-11 18:19:14 +08:00)


In [5]:
# Create a simulation for each analysis
simulation_1a = copy.deepcopy(time_domain_analysis.experiment.simulations[0])
simulation_1b = copy.deepcopy(cumulative_yield_analysis.experiment.simulations[0])
simulation_1c = copy.deepcopy(cumulative_yield_analysis.experiment.simulations[0])

time: 12.6 ms (started: 2023-01-11 18:19:14 +08:00)


In [6]:
# Experiment configuration
simulation_1a.model.initial_state.update({})

simulation_1a.model.params.update({})

time: 11.3 ms (started: 2023-01-11 18:19:14 +08:00)


# Analysis 1: ...

{Analysis Description}

In [7]:
# Analysis-specific setup
normal_adoption = simulation_1a.model.params['validator_process'][0](_run=None, _timestep=None)
simulation_1a.model.params.update({
    'validator_process': [
        lambda _run, _timestep: normal_adoption,  # Normal adoption: current 6-month average active validators per epoch from The Graph Subgraph
        lambda _run, _timestep: normal_adoption * 0.5,  # Low adoption: 50%-lower scenario
        lambda _run, _timestep: normal_adoption * 1.5,  # High adoption: 50%-higher scenario
    ],  # New validators per epoch
})

time: 11.2 ms (started: 2023-01-11 18:19:14 +08:00)


In [8]:
# Experiment execution
df_1a, _exceptions = run(simulation_1a)

2023-01-11 18:19:15,015 - root - INFO - Running experiment
2023-01-11 18:19:15,017 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-01-11 18:19:15,078 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-01-11 18:19:15,143 - root - INFO - Starting simulation 0 / run 0 / subset 2
2023-01-11 18:19:15,203 - root - INFO - Experiment complete in 0.18674087524414062 seconds
2023-01-11 18:19:15,203 - root - INFO - Post-processing results
2023-01-11 18:19:15,717 - root - INFO - Post-processing complete in 0.5145320892333984 seconds
time: 714 ms (started: 2023-01-11 18:19:15 +08:00)


In [9]:
print(df_1a[["timestamp","subset","number_of_awake_validators","number_of_active_validators","average_effective_balance",'network_issuance','polygn_supply','polygn_staked','supply_inflation']])
for i in df_1a.columns:
    print(i)

                      timestamp  subset  number_of_awake_validators  \
1    2023-01-11 18:18:47.976053       0                         100   
2    2023-01-12 18:18:47.976053       0                         100   
3    2023-01-13 18:18:47.976053       0                         100   
4    2023-01-14 18:18:47.976053       0                         100   
5    2023-01-15 18:18:47.976053       0                         100   
...                         ...     ...                         ...   
3238 2025-12-21 18:18:47.976053       2                         100   
3239 2025-12-22 18:18:47.976053       2                         100   
3240 2025-12-23 18:18:47.976053       2                         100   
3241 2025-12-24 18:18:47.976053       2                         100   
3242 2025-12-25 18:18:47.976053       2                         100   

      number_of_active_validators  average_effective_balance  \
1                             100               3.000000e+16   
2                  

In [10]:
# Post-processing and visualizations
visualizations.plot_number_of_validators_per_subset(df_1a, scenario_names={0: "Normal Adoption", 1: "Low Adoption", 2: "High Adoption"})

time: 175 ms (started: 2023-01-11 18:19:15 +08:00)


In [11]:
visualizations.plot_yields_per_subset_subplots(
    df_1a,
    subplot_titles=['Normal Adoption', 'Low Adoption', 'High Adoption']
)

time: 358 ms (started: 2023-01-11 18:19:15 +08:00)


In [12]:
visualizations.plot_yields_per_subset(df_1a, scenario_names={0: "Normal Adoption", 1: "Low Adoption", 2: "High Adoption"})

time: 228 ms (started: 2023-01-11 18:19:16 +08:00)


In [13]:
visualizations.plot_cumulative_yields_per_subset(df_1a, simulation_1a.model.params["dt"][0], scenario_names={0: "Normal Adoption", 1: "Low Adoption", 2: "High Adoption"})

time: 220 ms (started: 2023-01-11 18:19:16 +08:00)


In [14]:
simulation_1b.model.params.update({
    'base_fee_process': [
        lambda _run, _timestep: 0,
        lambda _run, _timestep: 30,
    ],
    'priority_fee_process': [
        lambda _run, _timestep: 0,
        lambda _run, _timestep: 2,
    ],
})

time: 14.5 ms (started: 2023-01-11 18:19:16 +08:00)


In [15]:
df_1b, _exceptions = run(simulation_1b)

2023-01-11 18:19:16,947 - root - INFO - Running experiment
2023-01-11 18:19:16,948 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-01-11 18:19:16,951 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-01-11 18:19:16,954 - root - INFO - Experiment complete in 0.0056819915771484375 seconds
2023-01-11 18:19:16,954 - root - INFO - Post-processing results
2023-01-11 18:19:16,998 - root - INFO - Post-processing complete in 0.04407906532287598 seconds
time: 64.3 ms (started: 2023-01-11 18:19:16 +08:00)


In [16]:
print(df_1b[['total_revenue_yields_pct','domain_treasury_balance_locked','private_treasury_balance','total_priority_fee_to_validators']])
for i in df_1b.columns:
    print(i)

    total_revenue_yields_pct  domain_treasury_balance_locked  \
1                   3.333333                    2.000000e+18   
2                   3.336071                    2.000000e+18   
3                   3.338811                    2.000000e+18   
4                   3.341554                    2.000000e+18   
5                   3.344298                    2.000000e+18   
..                       ...                             ...   
69                  3.424521                    2.003421e+18   
70                  3.427330                    2.003528e+18   
71                  3.430141                    2.003635e+18   
72                  3.432954                    2.003742e+18   
73                  3.435769                    2.003849e+18   

   private_treasury_balance total_priority_fee_to_validators  
1                     [0.0]                              0.0  
2                     [0.0]                              0.0  
3                     [0.0]               

In [17]:
# Calculate stacked revenue yields - subtract one yield value from the next, starting with largest yield scenario
df = df_1b.copy()
subsets = list(reversed(df.subset.unique()))
for subset in subsets:
    df.loc[df.subset == 0, 'cumulative_revenue_yields_stacked'] = df[df.subset == 0]['cumulative_revenue_yields_pct']
    if subset == 0:
        pass
    else:
        df.loc[df.subset == subset, 'cumulative_revenue_yields_stacked'] = \
            df[df.subset == subset]['cumulative_revenue_yields_pct'] - df[df.subset == subset - 1]['cumulative_revenue_yields_pct'].values

time: 23.9 ms (started: 2023-01-11 18:19:17 +08:00)


In [18]:
fig = visualizations.plot_stacked_cumulative_column_per_subset(
    df, column='cumulative_revenue_yields_stacked',
    scenario_names={0: 'PoS Issuance', 1: 'EIP-1559 Priority Fees'}
)

fig.update_layout(
    title={
        "text":"""
            Cumulative Revenue Yields: PoS Issuance, EIP-1559 Priority Fees
        """,
    },
    yaxis_title="Cumulative Revenue Yields (%)",
)

fig.show()

time: 28.4 ms (started: 2023-01-11 18:19:17 +08:00)


In [19]:
simulation_1c.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 2,
    "PRIVATE_CHAINS_CNT": 2,
})

simulation_1c.model.params.update({
    'base_fee_process': [
        lambda _run, _timestep: 0,
        lambda _run, _timestep: 30,
    ],
    'priority_fee_process': [
        lambda _run, _timestep: 0,
        lambda _run, _timestep: 2,
    ],
})

time: 15 ms (started: 2023-01-11 18:19:17 +08:00)


In [20]:
df_1c, _exceptions = run(simulation_1c)

2023-01-11 18:19:17,217 - root - INFO - Running experiment
2023-01-11 18:19:17,217 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-01-11 18:19:17,220 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-01-11 18:19:17,223 - root - INFO - Experiment complete in 0.005666017532348633 seconds
2023-01-11 18:19:17,223 - root - INFO - Post-processing results
2023-01-11 18:19:17,265 - root - INFO - Post-processing complete in 0.04198288917541504 seconds
time: 63.6 ms (started: 2023-01-11 18:19:17 +08:00)


In [21]:
print(df_1c[['total_revenue_yields_pct','domain_treasury_balance_locked','private_treasury_balance','total_priority_fee_to_validators']])

    total_revenue_yields_pct  domain_treasury_balance_locked  \
1                   3.333333                    2.000000e+18   
2                   3.336071                    2.000000e+18   
3                   3.338811                    2.000000e+18   
4                   3.341554                    2.000000e+18   
5                   3.344298                    2.000000e+18   
..                       ...                             ...   
69                  3.429781                    2.006843e+18   
70                  3.432589                    2.007057e+18   
71                  3.435400                    2.007271e+18   
72                  3.438213                    2.007484e+18   
73                  3.441029                    2.007698e+18   

                    private_treasury_balance total_priority_fee_to_validators  
1                                 [0.0, 0.0]                              0.0  
2                                 [0.0, 0.0]                           

In [22]:
# Calculate stacked revenue yields - subtract one yield value from the next, starting with largest yield scenario
df = df_1c.copy()
subsets = list(reversed(df.subset.unique()))
for subset in subsets:
    df.loc[df.subset == 0, 'cumulative_revenue_yields_stacked'] = df[df.subset == 0]['cumulative_revenue_yields_pct']
    if subset == 0:
        pass
    else:
        df.loc[df.subset == subset, 'cumulative_revenue_yields_stacked'] = \
            df[df.subset == subset]['cumulative_revenue_yields_pct'] - df[df.subset == subset - 1]['cumulative_revenue_yields_pct'].values

time: 15.7 ms (started: 2023-01-11 18:19:17 +08:00)


In [23]:
subsets

[1, 0]

time: 14.1 ms (started: 2023-01-11 18:19:17 +08:00)


In [24]:
fig = visualizations.plot_stacked_cumulative_column_per_subset(
    df, column='cumulative_revenue_yields_stacked',
    scenario_names={0: 'PoS Issuance', 1: 'EIP-1559 Priority Fees'}
)

fig.update_layout(
    title={
        "text":"""
            Cumulative Revenue Yields: PoS Issuance, EIP-1559 Priority Fees
        """,
    },
    yaxis_title="Cumulative Revenue Yields (%)",
)

fig.show()

time: 25.9 ms (started: 2023-01-11 18:19:17 +08:00)


In [25]:
visualizations.plot_treasury_per_subset(
    df_1c,
    scenario_names=['PoS Issuance', 'EIP-1559']
)

time: 24.6 ms (started: 2023-01-11 18:19:17 +08:00)


In [26]:
# gas cost comsumption and validator yield analysis
simulation_2 = copy.deepcopy(cumulative_yield_analysis.experiment.simulations[0])

time: 13.7 ms (started: 2023-01-11 18:19:17 +08:00)


In [27]:
simulation_2.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 5,
    "PRIVATE_CHAINS_CNT": 5,
})

PoS_checkpoint_gas_cost = simulation_2.model.params['checkpoint_gas_cost'][0]
simulation_2.model.params.update({
    'checkpoint_gas_cost': [
        1/100 * PoS_checkpoint_gas_cost,
        #10/100 * PoS_checkpoint_gas_cost,
        #50/100 * PoS_checkpoint_gas_cost,
        1 * PoS_checkpoint_gas_cost,
        1 * PoS_checkpoint_gas_cost,
    ],
    'epochs_once_a_checkpoint_submission': [
        1,
        1,
        5,
    ],
    
})

time: 13.4 ms (started: 2023-01-11 18:19:17 +08:00)


In [28]:
df_2, _exceptions = run(simulation_2)

2023-01-11 18:19:17,545 - root - INFO - Running experiment
2023-01-11 18:19:17,546 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-01-11 18:19:17,549 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-01-11 18:19:17,552 - root - INFO - Starting simulation 0 / run 0 / subset 2
2023-01-11 18:19:17,554 - root - INFO - Experiment complete in 0.008761167526245117 seconds
2023-01-11 18:19:17,555 - root - INFO - Post-processing results
2023-01-11 18:19:17,607 - root - INFO - Post-processing complete in 0.0526728630065918 seconds
time: 76.1 ms (started: 2023-01-11 18:19:17 +08:00)


In [29]:
visualizations.plot_yields_per_subset_subplots(
    df_2,
    subplot_titles=['0.1 normal gas + 225 checkpoints per day', '1 normal gas + 225 checkpoints per day', '1 normal gas + 45 checkpoints per day']
)

time: 45.6 ms (started: 2023-01-11 18:19:17 +08:00)


In [30]:
visualizations.plot_yields_per_subset(df_2, scenario_names={0: '0.01 normal gas + 225 checkpoints per day', 1: '1 normal gas + 225 checkpoints per day', 2: '1 normal gas + 45 checkpoints per day'})

time: 33.1 ms (started: 2023-01-11 18:19:17 +08:00)


In [42]:
df_2b[["subset","validator_checkpoint_costs"]]

,subset,validator_checkpoint_costs
1,0,510.3
2,0,510.3
3,0,510.3
4,0,510.3
5,0,510.3
...,...,...
106,2,51030.0
107,2,51030.0
108,2,51030.0
109,2,51030.0


time: 17.6 ms (started: 2023-01-11 18:24:26 +08:00)


In [32]:
visualizations.plot_cumulative_yields_per_subset(df_2, simulation_2.model.params["dt"][0], scenario_names={0: '0.1 normal gas + 225 checkpoints per day', 1: '1 normal gas + 225 checkpoints per day', 2: '1 normal gas + 45 checkpoints per day'})

time: 36.2 ms (started: 2023-01-11 18:19:17 +08:00)


In [4]:
# gas cost comsumption and validator yield analysis
simulation_2b = copy.deepcopy(cumulative_yield_analysis.experiment.simulations[0])

time: 13.6 ms (started: 2023-01-12 18:37:34 +08:00)


In [5]:
simulation_2b.model.initial_state.update({
    "PUBLIC_CHAINS_CNT": 500,
    "PRIVATE_CHAINS_CNT": 500,
})

PoS_checkpoint_gas_cost = simulation_2b.model.params['checkpoint_gas_cost'][0]
simulation_2b.model.params.update({
    'checkpoint_gas_cost': [
        1/100 * PoS_checkpoint_gas_cost,
        #10/100 * PoS_checkpoint_gas_cost,
        #50/100 * PoS_checkpoint_gas_cost,
        1 * PoS_checkpoint_gas_cost,
        1 * PoS_checkpoint_gas_cost,
    ],
    ' epochs_once_a_checkpoint_submission': [
        1,
        1,
        5,
    ],
    
})

time: 11.6 ms (started: 2023-01-12 18:37:35 +08:00)


In [6]:
df_2b, _exceptions = run(simulation_2b)

2023-01-12 18:37:36,051 - root - INFO - Running experiment
2023-01-12 18:37:36,053 - root - INFO - Starting simulation 0 / run 0 / subset 0
2023-01-12 18:37:36,057 - root - INFO - Starting simulation 0 / run 0 / subset 1
2023-01-12 18:37:36,061 - root - INFO - Starting simulation 0 / run 0 / subset 2
2023-01-12 18:37:36,065 - root - INFO - Experiment complete in 0.01380300521850586 seconds
2023-01-12 18:37:36,066 - root - INFO - Post-processing results
2023-01-12 18:37:36,120 - root - INFO - Post-processing complete in 0.05453801155090332 seconds
time: 81 ms (started: 2023-01-12 18:37:36 +08:00)


In [7]:
visualizations.plot_yields_per_subset_subplots(
    df_2b,
    subplot_titles=['0.01 normal gas + 225 checkpoints per day', '1 normal gas + 225 checkpoints per day', '1 normal gas + 45 checkpoints per day']
)

time: 139 ms (started: 2023-01-12 18:37:36 +08:00)


In [9]:
visualizations.plot_yields_per_subset(df_2b, scenario_names={0: '0.01 normal gas + 225 checkpoints per day', 1: '1 normal gas + 225 checkpoints per day', 2: '1 normal gas + 45 checkpoints per day'})

time: 44.7 ms (started: 2023-01-12 18:35:08 +08:00)
